In [8]:
import requests
import json
from datetime import datetime
import threading
from http.server import HTTPServer, BaseHTTPRequestHandler
import webbrowser
from urllib.parse import urlparse, parse_qs
import yaml

CLIENT_ID = None
CLIENT_SECRET = None

AUTH_CODE = None
ACCESS_TOKEN = None
REFRESH_TOKEN = None

class MyHandler(BaseHTTPRequestHandler):

    def do_GET(self):
        global AUTH_CODE
        
        query_components = parse_qs(urlparse(self.path).query)
        AUTH_CODE = query_components['code']
        self.send_response(200)
        self.end_headers()
        self.wfile.write(str.encode('Received auth code: {}'.format(AUTH_CODE)))
        
        # shutting down
        self.server.running = False

        
class MainServer:
    def __init__(self, port = 8123):
        self._server = HTTPServer(('0.0.0.0', port), MyHandler)
        self._thread = threading.Thread(target=self.run)
        self._thread.deamon = True 

    def run(self):
        self._server.running = True
        while self._server.running:
            self._server.handle_request()

    def start(self):
        self._thread.start()

    def shut_down(self):
        self._thread.close()
        

def reset_tokens():
    global AUTH_CODE, ACCESS_TOKEN, REFRESH_TOKEN
    AUTH_CODE = None
    ACCESS_TOKEN = None
    REFRESH_TOKEN = None


def setup(client_id=None, client_secret=None):
    global CLIENT_ID, CLIENT_SECRET
    
    if client_id is None and client_secret is None:
        with open('../credentials') as file:
            creds = yaml.full_load(file)
            CLIENT_ID = creds['client_id']
            CLIENT_SECRET = creds['client_secret']
    else:
        CLIENT_ID = client_id
        CLIENT_SECRET = client_secret
    
    reset_tokens()
    
    server = MainServer()
    server.start()
    webbrowser.open('https://www.strava.com/api/v3/oauth/authorize?response_type=code&client_id={}&redirect_uri=http://localhost:8123/give-me-token&scope=activity:read_all,profile:read_all&approval_prompt=force'.format(CLIENT_ID))
    

def refresh_token():
    global ACCESS_TOKEN, REFRESH_TOKEN
    url = 'https://www.strava.com/api/v3/oauth/token'
    
    payload = {
        'client_id': CLIENT_ID,
        'client_secret': CLIENT_SECRET,
        'grant_type': 'authorization_code' if REFRESH_TOKEN is None else 'refresh_token',
        'code': AUTH_CODE if REFRESH_TOKEN is None else REFRESH_TOKEN
    }
    
    headers = {
      'Content-Type': 'application/x-www-form-urlencoded'
    }
    
    print(payload)

    response = requests.request("POST", url, headers=headers, data = payload)
    
    if response.status_code > 299:
        raise Exception(response.status_code, response.text)
    
    parsed_response = json.loads(response.text)
    ACCESS_TOKEN = parsed_response['access_token']
    REFRESH_TOKEN = parsed_response['refresh_token']
    
    print(parsed_response)
    
    return 'expires_at={}'.format(datetime.fromtimestamp(parsed_response['expires_at']))



def api_request(endpoint, retries=3, method='GET'):
    url = "https://www.strava.com/api/v3" + endpoint

    payload = {}
    headers = {
      'Authorization': 'Bearer {}'.format(ACCESS_TOKEN)
    }

    response = requests.request(method, url, headers=headers, data = payload)
    
    if response.status_code > 299:
        raise Exception(response.status_code, response.text)
    
    return json.loads(response.text)


In [9]:
"""
Setup authorization from credentials file
"""
setup()

127.0.0.1 - - [06/Mar/2020 16:17:18] "GET /give-me-token?state=&code=9ab1fe8f12cbdeaa9fdafc539d1093ecef7f4757&scope=read,activity:read_all,profile:read_all HTTP/1.1" 200 -


In [10]:
"""
Refresh access token
"""
refresh_token()

{'client_id': 20539, 'client_secret': '954df898cdd4d90a239040d2124aa2a6afceab41', 'grant_type': 'authorization_code', 'code': ['9ab1fe8f12cbdeaa9fdafc539d1093ecef7f4757']}
{'token_type': 'Bearer', 'expires_at': 1583529320, 'expires_in': 21478, 'refresh_token': '5f2e8b899c82e4b28b93a0fb9c34aee6c2a78074', 'access_token': 'ae01846f38c055ca4890444607557a807c74a672', 'athlete': {'id': 2706822, 'username': 'mat_kuz', 'resource_state': 2, 'firstname': 'Mateusz', 'lastname': 'Kuźmik', 'city': 'Rzeszów', 'state': 'Województwo podkarpackie', 'country': 'Polska', 'sex': 'M', 'premium': True, 'summit': True, 'created_at': '2013-08-01T12:38:14Z', 'updated_at': '2020-02-07T13:00:06Z', 'badge_type_id': 1, 'profile_medium': 'https://dgalywyr863hv.cloudfront.net/pictures/athletes/2706822/13107580/1/medium.jpg', 'profile': 'https://dgalywyr863hv.cloudfront.net/pictures/athletes/2706822/13107580/1/large.jpg', 'friend': None, 'follower': None}}


'expires_at=2020-03-06 22:15:20'

In [36]:
# stream types: "watts" | "heartrate"

series_response = api_request('/activities/3114845146/streams?keys=watts,heartrate,time')

list(map(lambda x: x['type'], series_response))

['heartrate', 'watts', 'distance', 'time']

In [39]:
import pandas as pd

frame_source = {x['data'] for x in series_response if x['type'] in ['heartrate', 'watts', 'time']}

[{'type': 'heartrate',
  'data': [99,
   99,
   102,
   102,
   104,
   103,
   104,
   105,
   106,
   107,
   108,
   108,
   108,
   107,
   106,
   106,
   104,
   104,
   103,
   105,
   104,
   106,
   106,
   106,
   104,
   105,
   106,
   106,
   107,
   107,
   107,
   107,
   108,
   107,
   107,
   108,
   108,
   108,
   109,
   109,
   109,
   110,
   110,
   111,
   112,
   113,
   114,
   114,
   115,
   115,
   116,
   116,
   116,
   116,
   115,
   114,
   114,
   114,
   114,
   115,
   114,
   114,
   114,
   113,
   113,
   113,
   113,
   112,
   112,
   112,
   111,
   112,
   111,
   112,
   111,
   111,
   112,
   111,
   111,
   111,
   111,
   112,
   112,
   112,
   112,
   112,
   111,
   111,
   110,
   110,
   112,
   112,
   111,
   112,
   112,
   111,
   112,
   112,
   113,
   114,
   113,
   113,
   113,
   113,
   113,
   113,
   113,
   113,
   114,
   113,
   113,
   113,
   113,
   112,
   112,
   112,
   116,
   115,
   113,
   110,
   110,
   